In [ ]:
# default_exp consistent_mc_dropout

In [ ]:
# hide
import blackhc.project.script
from nbdev.showdoc import *

Appended /home/blackhc/PycharmProjects/bald-ical/src to paths
Switched to directory /home/blackhc/PycharmProjects/bald-ical
%load_ext autoreload
%autoreload 2


# Consistent MC Dropout
> Custom consistent dropout modules

For BNNs, we are going to use MC dropout.

To be able to compute BatchBALD scores, we need consistent MC dropout, which uses the consistent masks for inference. That means, that we draw $K$ masks and then keep them fixed while drawing the $K$ inference samples for each input in the test set.

During training, masks are redrawn for every sample.

In [ ]:
# export
from dataclasses import dataclass
from functools import wraps
from typing import List

import numpy as np
import torch
from blackhc.progress_bar import create_progress_bar
from toma import toma
from torch.nn import Module

## Bayesian Module

To make this work in an efficient way, we are going to define an abstract wrapper module that takes a batch `input_B` and outputs `results_B_K`.

Internally, it will blow up the input batch to $(B \cdot K) \times \cdots$ and then pass it to `mc_forward_impl`, which should be overriden.

`ConsistentMCDropout` layers will know to reshape the inputs to $B \times K \times \cdots$ and apply consistent masks.

In [ ]:
# exports
from torch.utils import data


@dataclass
class PredictionsLabels:
    predictions: torch.Tensor
    labels: torch.Tensor


class BayesianModule(Module):
    """A module that we can sample multiple times from given a single input batch.

    To be efficient, the module allows for a part of the forward pass to be deterministic.

    If we sample with "0" samples, we do a single deterministic forward pass with disabled dropout during evaluation.
    """

    # num MC dropout samples
    k = None

    def __init__(self):
        super().__init__()

    # Returns B x K x ...
    def forward(self, input_B: torch.Tensor, k: int):
        BayesianModule.k = k

        if k == 0:
            # No MC dropout (0 samples).
            features_B = self.deterministic_forward_impl(input_B)
            output_B = self.mc_forward_impl(features_B)
            output_B_1 = BayesianModule.unflatten_tensor(output_B, 1)
            return output_B_1

        features_B = self.deterministic_forward_impl(input_B)
        mc_features_BK = BayesianModule.mc_tensor(features_B, k)
        mc_output_BK = self.mc_forward_impl(mc_features_BK)
        mc_output_B_K = BayesianModule.unflatten_tensor(mc_output_BK, k)
        return mc_output_B_K

    def deterministic_forward_impl(self, input_B: torch.Tensor) -> torch.Tensor:
        return input_B

    def mc_forward_impl(self, mc_features_BK: torch.Tensor) -> torch.Tensor:
        return mc_features_BK

    @staticmethod
    def unflatten_tensor(input: torch.Tensor, k: int):
        input = input.view([-1, k] + list(input.shape[1:]))
        return input

    @staticmethod
    def flatten_tensor(mc_input: torch.Tensor):
        return mc_input.flatten(0, 1)

    @staticmethod
    def mc_tensor(input: torch.tensor, k: int):
        mc_shape = [input.shape[0], k] + list(input.shape[1:])
        return input.unsqueeze(1).expand(mc_shape).flatten(0, 1)

    @torch.no_grad()
    def _get_no_dropout_predictions_labels(self, *, loader: data.DataLoader, device):
        self.to(device=device)
        self.eval()

        N = len(loader.dataset)
        predictions = None
        labels = None

        pbar = create_progress_bar(N, tqdm_args=dict(desc="get_predictions_labels", leave=False))
        pbar.start()

        data_start = 0

        for batch_x, batch_labels in loader:
            batch_x = batch_x.to(device=device)
            batch_predictions = self(batch_x, k=0)

            batch_size = len(batch_predictions)
            data_end = data_start + batch_size

            # Support multi-dim labels.
            if labels is None:
                labels_shape = (N, *batch_labels.shape[1:])
                labels = torch.empty(labels_shape, dtype=batch_labels.dtype, device="cpu")
            # Support multi-dim predictions.
            if predictions is None:
                predictions_shape = (N, *batch_predictions.shape[1:])
                predictions = torch.empty(predictions_shape, dtype=batch_predictions.dtype, device="cpu")

            predictions[data_start:data_end].copy_(batch_predictions.float(), non_blocking=True)
            labels[data_start:data_end].copy_(batch_labels, non_blocking=True)

            data_start = data_end

            pbar.update(batch_size)

        pbar.finish()

        return predictions, labels

    @torch.no_grad()
    def get_predictions_labels(self, *, k: int, loader: data.DataLoader, device):
        assert_no_shuffling_no_augmentations_dataloader(loader)

        if k == 0:
            return self._get_no_dropout_predictions_labels(loader=loader, device=device)

        self.to(device=device)

        N = len(loader.dataset)
        predictions = None
        labels = None

        pbar = create_progress_bar(N * k, tqdm_args=dict(desc="get_predictions_labels", leave=False))
        pbar.start()

        @toma.execute.range(0, k, 128)
        def get_prediction_batch(start, end):
            nonlocal predictions
            nonlocal labels

            if start == 0:
                pbar.reset()

            self.eval()

            num_sub_samples = end - start

            data_start = 0
            # TODO: ensure that the dataloader is not shuffling!
            for batch_x, batch_labels in loader:
                batch_x = batch_x.to(device=device)
                batch_predictions = self(batch_x, num_sub_samples)

                batch_size = len(batch_predictions)
                data_end = data_start + batch_size

                # Support multi-dim predictions.
                if predictions is None:
                    predictions_shape = (N, *batch_predictions.shape[1:])
                    predictions = torch.empty(predictions_shape, dtype=batch_predictions.dtype, device="cpu")
                # Support multi-dim labels.
                if labels is None:
                    labels_shape = (N, *batch_labels.shape[1:])
                    labels = torch.empty(labels_shape, dtype=batch_labels.dtype, device="cpu")

                predictions[data_start:data_end, start:end].copy_(batch_predictions.float(), non_blocking=True)
                if start == 0:
                    labels[data_start:data_end].copy_(batch_labels, non_blocking=True)

                data_start = data_end

                pbar.update(batch_size * num_sub_samples)

        pbar.finish()

        return predictions, labels

NameError: name 'Module' is not defined

## Consistent MC Dropout

In [ ]:
# exports


class _ConsistentMCDropout(Module):
    def __init__(self, p=0.5):
        super().__init__()

        if p < 0 or p > 1:
            raise ValueError("dropout probability has to be between 0 and 1, " "but got {}".format(p))

        self.p = p
        self.mask = None

    def extra_repr(self):
        return "p={}".format(self.p)

    def reset_mask(self):
        self.mask = None

    def train(self, mode=True):
        super().train(mode)
        if not mode:
            self.reset_mask()

    def _get_sample_mask_shape(self, sample_shape):
        return sample_shape

    def _create_mask(self, input, k):
        mask_shape = [1, k] + list(self._get_sample_mask_shape(input.shape[1:]))
        mask = torch.empty(mask_shape, dtype=torch.bool, device=input.device).bernoulli_(self.p)
        return mask

    def forward(self, input: torch.Tensor):
        if self.p == 0.0:
            return input

        k = BayesianModule.k

        # Disable dropout during evaluation if k=0 (i.e. no samples).
        if not self.training and k == 0:
            return input

        if self.training:
            # Create a new mask on each call and for each batch element.
            k = input.shape[0] * max(k, 1)
            mask = self._create_mask(input, k)
        else:
            if self.mask is None:
                # print('recreating mask', self)
                # Recreate mask.
                self.mask = self._create_mask(input, k)

            mask = self.mask

        mc_input = BayesianModule.unflatten_tensor(input, k)
        mc_output = mc_input.masked_fill(mask, 0) / (1 - self.p)

        # Flatten MCDI, batch into one dimension again.
        return BayesianModule.flatten_tensor(mc_output)

In [ ]:
# export


class ConsistentMCDropout(_ConsistentMCDropout):
    r"""Randomly zeroes some of the elements of the input
    tensor with probability :attr:`p` using samples from a Bernoulli
    distribution. The elements to zero are randomized on every forward call during training time.

    During eval time, a fixed mask is picked and kept until `reset_mask()` is called.

    This has proven to be an effective technique for regularization and
    preventing the co-adaptation of neurons as described in the paper
    `Improving neural networks by preventing co-adaptation of feature
    detectors`_ .

    Furthermore, the outputs are scaled by a factor of :math:`\frac{1}{1-p}` during
    training. This means that during evaluation the module simply computes an
    identity function.

    Args:
        p: probability of an element to be zeroed. Default: 0.5
        inplace: If set to ``True``, will do this operation in-place. Default: ``False``

    Shape:
        - Input: `Any`. Input can be of any shape
        - Output: `Same`. Output is of the same shape as input

    Examples::

        >>> m = nn.Dropout(p=0.2)
        >>> input = torch.randn(20, 16)
        >>> output = m(input)

    .. _Improving neural networks by preventing co-adaptation of feature
        detectors: https://arxiv.org/abs/1207.0580
    """
    pass


class ConsistentMCDropout2d(_ConsistentMCDropout):
    r"""Randomly zeroes whole channels of the input tensor.
    The channels to zero-out are randomized on every forward call.

    During eval time, a fixed mask is picked and kept until `reset_mask()` is called.

    Usually the input comes from :class:`nn.Conv2d` modules.

    As described in the paper
    `Efficient Object Localization Using Convolutional Networks`_ ,
    if adjacent pixels within feature maps are strongly correlated
    (as is normally the case in early convolution layers) then i.i.d. dropout
    will not regularize the activations and will otherwise just result
    in an effective learning rate decrease.

    In this case, :func:`nn.Dropout2d` will help promote independence between
    feature maps and should be used instead.

    Args:
        p (float, optional): probability of an element to be zero-ed.
        inplace (bool, optional): If set to ``True``, will do this operation
            in-place

    Shape:
        - Input: :math:`(N, C, H, W)`
        - Output: :math:`(N, C, H, W)` (same shape as input)

    Examples::

        >>> m = nn.Dropout2d(p=0.2)
        >>> input = torch.randn(20, 16, 32, 32)
        >>> output = m(input)

    .. _Efficient Object Localization Using Convolutional Networks:
       http://arxiv.org/abs/1411.4280
    """

    def _get_sample_mask_shape(self, sample_shape):
        return [sample_shape[0]] + [1] * (len(sample_shape) - 1)

## Example

The following defines a DNN module that can learn MNIST.

In [ ]:
import torch
from torch import nn as nn
from torch.nn import functional as F


class BayesianCNN(BayesianModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor):
        input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
        input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
        input = input.view(-1, 1024)
        input = F.relu(self.fc1_drop(self.fc1(input)))
        input = self.fc2(input)
        input = F.log_softmax(input, dim=1)

        return input


BayesianCNN()

BayesianCNN(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv1_drop): ConsistentMCDropout2d(p=0.5)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): ConsistentMCDropout2d(p=0.5)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc1_drop): ConsistentMCDropout(p=0.5)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

## Additional wrappers

To make it easier for training in inference and training with regular frameworks, we want to keep k fixed.

In [ ]:
# exports


class SamplerModel(Module):
    """Wrap a `BayesianModule` to sample k MC dropout samples consistently.

    A forward pass returns: log probs BxKxC for a batch of B inputs, K MC dropout samples, and C classes.
    """

    def __init__(self, bayesian_module: BayesianModule, k: int):
        super().__init__()
        self.bayesian_module = bayesian_module
        self.k = k

    def forward(self, input: torch.Tensor):
        log_probs_B_K_C = self.bayesian_module(input, self.k)
        return log_probs_B_K_C

For better Dropout optimization, we sometimes want to use multiple samples that we train independently (Decoder Cross-Entropy):

In [ ]:
# exports


def multi_sample_loss(loss):
    """Wrap a loss function to expand the targets to work together with a SamplerModel."""
    @wraps(loss)
    def wrapped_loss(input_B_K_C, target_B_, *args, **kwargs):
        assert input_B_K_C.shape[0] == target_B_.shape[0]
        input_BK_C = input_B_K_C.flatten(0, 1)
        target_B_K_ = target_B_[:, None].expand(input_B_K_C.shape[:2] + target_B_.shape[1:])
        target_BK_ = target_B_K_.flatten(0, 1)
        return loss(input_BK_C, target_BK_, *args, **kwargs)

    return wrapped_loss

In [ ]:
fake_inputs = torch.arange(100).reshape(2, 5, 10).float()
fake_target = torch.arange(2)

loss = torch.nn.CrossEntropyLoss()

wrapped_loss = multi_sample_loss(loss)

wrapped_loss(fake_inputs, fake_target)

tensor(8.9586)

For final test predictions, we want to take the geometric mean (average of logits):

In [ ]:
# exports


def geometric_mean_loss(loss):
    @wraps(loss)
    def wrapped_loss(log_probs_B_K_C, target_N, *args, **kwargs):
        return loss(log_probs_B_K_C.mean(dim=1, keepdim=False), target_N, *args, **kwargs)

    return wrapped_loss


class GeometricMeanPrediction(Module):
    """
    Use a eg. SamplerModel and compute the geometric mean as "ensemble" prediction.

    We assume we receive log probs (so after Softmax).
    """

    def __init__(self, module: Module):
        super().__init__()
        self.module = module

    def forward(self, input: torch.Tensor):
        log_probs_B_K_C = self.module(input)
        return log_probs_B_K_C.mean(dim=1, keepdim=False)


class LogProbMeanPrediction(Module):
    """
    Use a eg. SamplerModel and compute the geometric mean as "ensemble" prediction.
    """

    def __init__(self, module: Module):
        super().__init__()
        self.module = module

    def forward(self, input: torch.Tensor):
        log_probs_B_K_C = self.module(input)
        log_mean_probs = log_probs_B_K_C.logsumexp(dim=1, keepdim=False) - np.log(log_probs_B_K_C.shape[1])
        return log_mean_probs


def get_log_mean_probs(log_probs_N_K_C):
    # Arithmetic mean of the probs (valid MC dropout)
    log_mean_probs_N_C = log_probs_N_K_C.logsumexp(dim=1, keepdim=False) - np.log(log_probs_N_K_C.shape[1])
    return log_mean_probs_N_C


def assert_no_shuffling_no_augmentations_dataloader(dataloader: data.DataLoader):
    batch_x_A = None
    batch_labels_A = None
    batch_x_B = None
    batch_labels_B = None

    for batch_x_A, batch_labels_A in dataloader:
        break

    for batch_x_B, batch_labels_B in dataloader:
        break

    assert batch_x_A == batch_x_B, "Batch inputs different. Augmentations enabled, or dataloader shuffles data?!"
    assert batch_labels_A == batch_labels_B, "Batch labels different. Augmentations enabled, or dataloader shuffles data?!"


def get_bayesian_ensemble_predictions_labels(*, modules: List[BayesianModule], k: int, loader: data.DataLoader, device):
    assert_no_shuffling_no_augmentations_dataloader(loader)

    ensemble_predictions = []
    ensemble_labels = None

    for module in modules:
        predictions, labels = module.get_predictions_labels(k=k, loader=loader, device=device)

        ensemble_predictions += [predictions]
        if ensemble_labels is not None:
            assert torch.all(ensemble_labels == labels)
        else:
            ensemble_labels = labels

        module.to("cpu")

    ensemble_predictions = torch.cat(ensemble_predictions, dim=1)
    return ensemble_predictions, ensemble_labels